In [1]:
import numpy as np
import pandas as pd
import os
import time
from IPython import display
import PIL
import torch

music_data_folder = 'Data/genres_original/'
image_data_folder = 'Data/images_original/'
features_30_seconds = "Data/features_30_sec.csv"
features_3_seconds = "Data/features_3_sec.csv"

In [2]:
import neuralnetworks as nn
import mlfuncs
from nnclassifier import NNClassifier


In [3]:
data_3 = pd.read_csv(features_3_seconds)

data_3

,filename,length,chroma_stft_mean,chroma_stft_var,rms_mean,rms_var,spectral_centroid_mean,spectral_centroid_var,spectral_bandwidth_mean,spectral_bandwidth_var,...,mfcc16_var,mfcc17_mean,mfcc17_var,mfcc18_mean,mfcc18_var,mfcc19_mean,mfcc19_var,mfcc20_mean,mfcc20_var,label
0,blues.00000.0.wav,66149,0.335406,0.091048,0.130405,0.003521,1773.065032,167541.630869,1972.744388,117335.771563,...,39.687145,-3.241280,36.488243,0.722209,38.099152,-5.050335,33.618073,-0.243027,43.771767,blues
1,blues.00000.1.wav,66149,0.343065,0.086147,0.112699,0.001450,1816.693777,90525.690866,2010.051501,65671.875673,...,64.748276,-6.055294,40.677654,0.159015,51.264091,-2.837699,97.030830,5.784063,59.943081,blues
2,blues.00000.2.wav,66149,0.346815,0.092243,0.132003,0.004620,1788.539719,111407.437613,2084.565132,75124.921716,...,67.336563,-1.768610,28.348579,2.378768,45.717648,-1.938424,53.050835,2.517375,33.105122,blues
3,blues.00000.3.wav,66149,0.363639,0.086856,0.132565,0.002448,1655.289045,111952.284517,1960.039988,82913.639269,...,47.739452,-3.841155,28.337118,1.218588,34.770935,-3.580352,50.836224,3.630866,32.023678,blues
4,blues.00000.4.wav,66149,0.335579,0.088129,0.143289,0.001701,1630.656199,79667.267654,1948.503884,60204.020268,...,30.336359,0.664582,45.880913,1.689446,51.363583,-3.392489,26.738789,0.536961,29.146694,blues
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9985,rock.00099.5.wav,66149,0.349126,0.080515,0.050019,0.000097,1499.083005,164266.886443,1718.707215,85931.574523,...,42.485981,-9.094270,38.326839,-4.246976,31.049839,-5.625813,48.804092,1.818823,38.966969,rock
9986,rock.00099.6.wav,66149,0.372564,0.082626,0.057897,0.000088,1847.965128,281054.935973,1906.468492,99727.037054,...,32.415203,-12.375726,66.418587,-3.081278,54.414265,-11.960546,63.452255,0.428857,18.697033,rock
9987,rock.00099.7.wav,66149,0.347481,0.089019,0.052403,0.000701,1346.157659,662956.246325,1561.859087,138762.841945,...,78.228149,-2.524483,21.778994,4.809936,25.980829,1.775686,48.582378,-0.299545,41.586990,rock
9988,rock.00099.8.wav,66149,0.387527,0.084815,0.066430,0.000320,2084.515327,203891.039161,2018.366254,22860.992562,...,28.323744,-5.363541,17.209942,6.462601,21.442928,2.354765,24.843613,0.675824,12.787750,rock


### Using the 3 seconds features dataset

In [4]:
cols = list(data_3.columns)
cols

['filename',
 'length',
 'chroma_stft_mean',
 'chroma_stft_var',
 'rms_mean',
 'rms_var',
 'spectral_centroid_mean',
 'spectral_centroid_var',
 'spectral_bandwidth_mean',
 'spectral_bandwidth_var',
 'rolloff_mean',
 'rolloff_var',
 'zero_crossing_rate_mean',
 'zero_crossing_rate_var',
 'harmony_mean',
 'harmony_var',
 'perceptr_mean',
 'perceptr_var',
 'tempo',
 'mfcc1_mean',
 'mfcc1_var',
 'mfcc2_mean',
 'mfcc2_var',
 'mfcc3_mean',
 'mfcc3_var',
 'mfcc4_mean',
 'mfcc4_var',
 'mfcc5_mean',
 'mfcc5_var',
 'mfcc6_mean',
 'mfcc6_var',
 'mfcc7_mean',
 'mfcc7_var',
 'mfcc8_mean',
 'mfcc8_var',
 'mfcc9_mean',
 'mfcc9_var',
 'mfcc10_mean',
 'mfcc10_var',
 'mfcc11_mean',
 'mfcc11_var',
 'mfcc12_mean',
 'mfcc12_var',
 'mfcc13_mean',
 'mfcc13_var',
 'mfcc14_mean',
 'mfcc14_var',
 'mfcc15_mean',
 'mfcc15_var',
 'mfcc16_mean',
 'mfcc16_var',
 'mfcc17_mean',
 'mfcc17_var',
 'mfcc18_mean',
 'mfcc18_var',
 'mfcc19_mean',
 'mfcc19_var',
 'mfcc20_mean',
 'mfcc20_var',
 'label']

In [5]:
T = data_3['label'].values.reshape(-1, 1)
X = data_3[cols[2:-1]].values

X.shape, T.shape

((9990, 57), (9990, 1))

In [6]:
Xtrain, Ttrain, Xvalidate, Tvalidate, Xtest, Ttest = next(mlfuncs.generate_partitions(X, T, 10, classification=True))

In [12]:
results_features_3 = []

In [13]:
n_fcs = [[], [50], [100, 50], [100, 500, 50], [100, 250, 250, 100]]
n_epochs = [1000]
n_lrs = [0.01, .005]
n_batch_sizes = [100, 200]
n_activations = ['tanh', 'relu']
device = torch.device('cuda:0')



nn_num = 1

for lr in n_lrs:
    for bs in n_batch_sizes:
        for epochs in n_epochs:
            for act in n_activations:
                for fc in n_fcs:
                    print('Network#{}'.format(nn_num), end='')
                    nn_num +=1

                    nnet = NNClassifier( Xtrain.shape[1], fc, len(np.unique(Ttrain)), activation_function=act, device=device)

                    nnet.train(Xtrain, Ttrain, epochs, batch_size=bs, learning_rate=lr, method='adam', verbose=False)
                    tr_ac = mlfuncs.percent_equal(nnet.use(Xtrain)[0], Ttrain)
                    val_ac = mlfuncs.percent_equal(nnet.use(Xvalidate)[0], Tvalidate)
                    te_ac = mlfuncs.percent_equal(nnet.use(Xtest)[0], Ttest)

                    result = [lr, bs, epochs, act, fc, tr_ac, val_ac, te_ac]
                    results_features_3.append(result)
                    print(result)

Network#1[0.01, 100, 1000, 'tanh', [], 74.58749999999999, 72.46231155778895, 72.06030150753769]
Network#2[0.01, 100, 1000, 'tanh', [50], 89.75, 75.07537688442211, 72.8643216080402]
Network#3[0.01, 100, 1000, 'tanh', [100, 50], 99.9, 83.71859296482413, 81.90954773869346]
Network#4[0.01, 100, 1000, 'tanh', [100, 500, 50], 93.7125, 82.1105527638191, 80.7035175879397]
Network#5[0.01, 100, 1000, 'tanh', [100, 250, 250, 100], 80.83749999999999, 71.75879396984925, 70.55276381909547]
Network#6[0.01, 100, 1000, 'relu', [], 74.5625, 72.36180904522614, 72.06030150753769]
Network#7[0.01, 100, 1000, 'relu', [50], 95.5, 82.51256281407035, 81.20603015075378]
Network#8[0.01, 100, 1000, 'relu', [100, 50], 99.3125, 86.53266331658291, 85.92964824120602]
Network#9[0.01, 100, 1000, 'relu', [100, 500, 50], 97.5125, 85.92964824120602, 85.02512562814071]
Network#10[0.01, 100, 1000, 'relu', [100, 250, 250, 100], 96.2, 84.321608040201, 82.8140703517588]
Network#11[0.01, 200, 1000, 'tanh', [], 74.6875, 73.266331

In [14]:
results_features_3_df = pd.DataFrame(results_features_3, columns=['lr', 'bs', 'epochs', 'activation', 'fc', 'train', 'validate', 'test'])

results_features_3_df

,lr,bs,epochs,activation,fc,train,validate,test
0,0.010,100,1000,tanh,[],74.5875,72.462312,72.060302
1,0.010,100,1000,tanh,[50],89.7500,75.075377,72.864322
2,0.010,100,1000,tanh,"[100, 50]",99.9000,83.718593,81.909548
3,0.010,100,1000,tanh,"[100, 500, 50]",93.7125,82.110553,80.703518
4,0.010,100,1000,tanh,"[100, 250, 250, 100]",80.8375,71.758794,70.552764
5,0.010,100,1000,relu,[],74.5625,72.361809,72.060302
6,0.010,100,1000,relu,[50],95.5000,82.512563,81.206030
7,0.010,100,1000,relu,"[100, 50]",99.3125,86.532663,85.929648
8,0.010,100,1000,relu,"[100, 500, 50]",97.5125,85.929648,85.025126
9,0.010,100,1000,relu,"[100, 250, 250, 100]",96.2000,84.321608,82.814070


In [15]:
results_features_3_df.to_csv('results_features_3.csv', index=False)

In [23]:
X_combined = X.reshape((999, 57*10))

In [45]:
T_combined = T.reshape((999,-1))[:, :1]

In [46]:
T_combined[:101]

array([['blues'],
       ['blues'],
       ['blues'],
       ['blues'],
       ['blues'],
       ['blues'],
       ['blues'],
       ['blues'],
       ['blues'],
       ['blues'],
       ['blues'],
       ['blues'],
       ['blues'],
       ['blues'],
       ['blues'],
       ['blues'],
       ['blues'],
       ['blues'],
       ['blues'],
       ['blues'],
       ['blues'],
       ['blues'],
       ['blues'],
       ['blues'],
       ['blues'],
       ['blues'],
       ['blues'],
       ['blues'],
       ['blues'],
       ['blues'],
       ['blues'],
       ['blues'],
       ['blues'],
       ['blues'],
       ['blues'],
       ['blues'],
       ['blues'],
       ['blues'],
       ['blues'],
       ['blues'],
       ['blues'],
       ['blues'],
       ['blues'],
       ['blues'],
       ['blues'],
       ['blues'],
       ['blues'],
       ['blues'],
       ['blues'],
       ['blues'],
       ['blues'],
       ['blues'],
       ['blues'],
       ['blues'],
       ['blues'],
       ['b

In [47]:
X_combined.shape, T_combined.shape

((999, 570), (999, 1))

In [48]:
Xtrain, Ttrain, Xvalidate, Tvalidate, Xtest, Ttest = next(mlfuncs.generate_partitions(X_combined, T_combined, 10, classification=True))

In [55]:
results_features_3_combined = []

In [56]:
n_fcs = [[], [50], [100, 50], [100, 500, 50], [100, 250, 250, 100], [256, 512, 1024, 1024, 512, 256, 128]]
n_epochs = [1000, 3000]
n_lrs = [0.01, .005, 0.1, 0.005]
n_batch_sizes = [100, 200, 300]
n_activations = ['tanh', 'relu']
device = torch.device('cuda:0')



nn_num = 1

for lr in n_lrs:
    for bs in n_batch_sizes:
        for epochs in n_epochs:
            for act in n_activations:
                for fc in n_fcs:
                    print('Network#{}'.format(nn_num), end='')
                    nn_num +=1

                    nnet = NNClassifier( Xtrain.shape[1], fc, len(np.unique(Ttrain)), activation_function=act, device=device)

                    nnet.train(Xtrain, Ttrain, epochs, batch_size=bs, learning_rate=lr, method='adam', verbose=False)
                    tr_ac = mlfuncs.percent_equal(nnet.use(Xtrain)[0], Ttrain)
                    val_ac = mlfuncs.percent_equal(nnet.use(Xvalidate)[0], Tvalidate)
                    te_ac = mlfuncs.percent_equal(nnet.use(Xtest)[0], Ttest)

                    result = [lr, bs, epochs, act, fc, tr_ac, val_ac, te_ac]
                    results_features_3_combined.append(result)
                    print(result)

Network#1[0.01, 100, 1000, 'tanh', [], 100.0, 72.72727272727273, 62.62626262626263]
Network#2[0.01, 100, 1000, 'tanh', [50], 100.0, 75.75757575757575, 62.62626262626263]
Network#3[0.01, 100, 1000, 'tanh', [100, 50], 100.0, 76.76767676767676, 69.6969696969697]
Network#4[0.01, 100, 1000, 'tanh', [100, 500, 50], 100.0, 74.74747474747475, 66.66666666666666]
Network#5[0.01, 100, 1000, 'tanh', [100, 250, 250, 100], 100.0, 75.75757575757575, 67.67676767676768]
Network#6[0.01, 100, 1000, 'tanh', [256, 512, 1024, 1024, 512, 256, 128], 63.54556803995006, 55.55555555555556, 59.59595959595959]
Network#7[0.01, 100, 1000, 'relu', [], 100.0, 71.71717171717171, 62.62626262626263]
Network#8[0.01, 100, 1000, 'relu', [50], 100.0, 79.7979797979798, 66.66666666666666]
Network#9[0.01, 100, 1000, 'relu', [100, 50], 100.0, 76.76767676767676, 69.6969696969697]
Network#10[0.01, 100, 1000, 'relu', [100, 500, 50], 100.0, 73.73737373737373, 68.68686868686868]
Network#11[0.01, 100, 1000, 'relu', [100, 250, 250, 100

Network#81[0.005, 100, 1000, 'relu', [100, 50], 100.0, 79.7979797979798, 69.6969696969697]
Network#82[0.005, 100, 1000, 'relu', [100, 500, 50], 100.0, 80.8080808080808, 68.68686868686868]
Network#83[0.005, 100, 1000, 'relu', [100, 250, 250, 100], 100.0, 75.75757575757575, 66.66666666666666]
Network#84[0.005, 100, 1000, 'relu', [256, 512, 1024, 1024, 512, 256, 128], 100.0, 75.75757575757575, 65.65656565656566]
Network#85[0.005, 100, 3000, 'tanh', [], 100.0, 68.68686868686868, 61.61616161616161]
Network#86[0.005, 100, 3000, 'tanh', [50], 100.0, 70.70707070707071, 62.62626262626263]
Network#87[0.005, 100, 3000, 'tanh', [100, 50], 100.0, 75.75757575757575, 63.63636363636363]
Network#88[0.005, 100, 3000, 'tanh', [100, 500, 50], 100.0, 76.76767676767676, 73.73737373737373]
Network#89[0.005, 100, 3000, 'tanh', [100, 250, 250, 100], 100.0, 73.73737373737373, 67.67676767676768]
Network#90[0.005, 100, 3000, 'tanh', [256, 512, 1024, 1024, 512, 256, 128], 100.0, 65.65656565656566, 61.6161616161616

Network#160[0.1, 100, 3000, 'tanh', [100, 500, 50], 57.927590511860174, 46.464646464646464, 44.44444444444444]
Network#161[0.1, 100, 3000, 'tanh', [100, 250, 250, 100], 43.07116104868914, 31.313131313131315, 26.262626262626267]
Network#162[0.1, 100, 3000, 'tanh', [256, 512, 1024, 1024, 512, 256, 128], 36.329588014981276, 27.27272727272727, 34.34343434343434]
Network#163[0.1, 100, 3000, 'relu', [], 100.0, 60.60606060606061, 48.484848484848484]
Network#164[0.1, 100, 3000, 'relu', [50], 100.0, 72.72727272727273, 67.67676767676768]
Network#165[0.1, 100, 3000, 'relu', [100, 50], 9.987515605493133, 10.1010101010101, 10.1010101010101]
Network#166[0.1, 100, 3000, 'relu', [100, 500, 50], 9.987515605493133, 10.1010101010101, 10.1010101010101]
Network#167[0.1, 100, 3000, 'relu', [100, 250, 250, 100], 10.112359550561797, 10.1010101010101, 10.1010101010101]
Network#168[0.1, 100, 3000, 'relu', [256, 512, 1024, 1024, 512, 256, 128], 9.987515605493133, 10.1010101010101, 10.1010101010101]
Network#169[0

Network#238[0.005, 100, 3000, 'relu', [100, 500, 50], 99.50062421972534, 69.6969696969697, 65.65656565656566]
Network#239[0.005, 100, 3000, 'relu', [100, 250, 250, 100], 100.0, 69.6969696969697, 64.64646464646465]
Network#240[0.005, 100, 3000, 'relu', [256, 512, 1024, 1024, 512, 256, 128], 9.987515605493133, 10.1010101010101, 10.1010101010101]
Network#241[0.005, 200, 1000, 'tanh', [], 100.0, 74.74747474747475, 63.63636363636363]
Network#242[0.005, 200, 1000, 'tanh', [50], 100.0, 78.78787878787878, 68.68686868686868]
Network#243[0.005, 200, 1000, 'tanh', [100, 50], 100.0, 79.7979797979798, 71.71717171717171]
Network#244[0.005, 200, 1000, 'tanh', [100, 500, 50], 100.0, 76.76767676767676, 69.6969696969697]
Network#245[0.005, 200, 1000, 'tanh', [100, 250, 250, 100], 100.0, 80.8080808080808, 70.70707070707071]
Network#246[0.005, 200, 1000, 'tanh', [256, 512, 1024, 1024, 512, 256, 128], 100.0, 58.58585858585859, 61.61616161616161]
Network#247[0.005, 200, 1000, 'relu', [], 100.0, 71.717171717

In [57]:
results_features_3_combined_df = pd.DataFrame(results_features_3_combined, columns=['lr', 'bs', 'epochs', 'activation', 'fc', 'train', 'validate', 'test'])

results_features_3_combined_df

,lr,bs,epochs,activation,fc,train,validate,test
0,0.010,100,1000,tanh,[],100.000000,72.727273,62.626263
1,0.010,100,1000,tanh,[50],100.000000,75.757576,62.626263
2,0.010,100,1000,tanh,"[100, 50]",100.000000,76.767677,69.696970
3,0.010,100,1000,tanh,"[100, 500, 50]",100.000000,74.747475,66.666667
4,0.010,100,1000,tanh,"[100, 250, 250, 100]",100.000000,75.757576,67.676768
...,...,...,...,...,...,...,...,...
283,0.005,300,3000,relu,[50],93.882647,78.787879,69.696970
284,0.005,300,3000,relu,"[100, 50]",93.882647,75.757576,70.707071
285,0.005,300,3000,relu,"[100, 500, 50]",94.257179,78.787879,71.717172
286,0.005,300,3000,relu,"[100, 250, 250, 100]",93.508115,78.787879,65.656566


In [58]:
results_features_3_combined_df.to_csv('results_features_3_combined.csv', index=False)

In [2]:
df = pd.read_csv('results_features_3.csv').sort_values(['validate', 'test'])

df

,lr,bs,epochs,activation,fc,train,validate,test
4,0.010,100,1000,tanh,"[100, 250, 250, 100]",80.8375,71.758794,70.552764
5,0.010,100,1000,relu,[],74.5625,72.361809,72.060302
20,0.005,100,1000,tanh,[],74.6250,72.361809,72.261307
0,0.010,100,1000,tanh,[],74.5875,72.462312,72.060302
25,0.005,100,1000,relu,[],74.6125,72.462312,72.160804
30,0.005,200,1000,tanh,[],74.5625,72.763819,72.261307
35,0.005,200,1000,relu,[],74.5625,72.763819,72.261307
15,0.010,200,1000,relu,[],74.7000,72.964824,72.160804
10,0.010,200,1000,tanh,[],74.6875,73.266332,72.160804
1,0.010,100,1000,tanh,[50],89.7500,75.075377,72.864322


In [3]:
df1 = pd.read_csv('results_features_3_combined.csv').sort_values(['validate', 'test'])

df1

,lr,bs,epochs,activation,fc,train,validate,test
176,0.100,200,1000,relu,"[100, 50]",10.112360,9.090909,8.080808
45,0.010,200,3000,relu,"[100, 500, 50]",10.112360,9.090909,9.090909
46,0.010,200,3000,relu,"[100, 250, 250, 100]",10.112360,9.090909,9.090909
119,0.005,200,3000,relu,"[256, 512, 1024, 1024, 512, 256, 128]",10.112360,9.090909,9.090909
177,0.100,200,1000,relu,"[100, 500, 50]",10.237203,9.090909,9.090909
...,...,...,...,...,...,...,...,...
249,0.005,200,1000,relu,"[100, 500, 50]",100.000000,80.808081,72.727273
247,0.005,200,1000,relu,[50],100.000000,82.828283,65.656566
128,0.005,300,1000,relu,"[100, 50]",93.757803,82.828283,68.686869
115,0.005,200,3000,relu,[50],100.000000,82.828283,71.717172


#### Running the best network

In [7]:
def percent_correct(Y, T):
    return np.mean(T == Y) * 100            


def confusion_matrix(Y_classes, T):
    class_names = np.unique(T)
    table = []
    for true_class in class_names:
        row = []
        for Y_class in class_names:
            row.append(100 * np.mean(Y_classes[T == true_class] == Y_class))
        table.append(row)
    conf_matrix = pd.DataFrame(table, index=class_names, columns=class_names)
    # cf.style.background_gradient(cmap='Blues').format("{:.1f} %")
    print('Percent Correct ', percent_correct(Y_classes, T))
    return conf_matrix.style.background_gradient(cmap='Blues').format("{:.1f}")

In [8]:
device = torch.device('cuda:0')

nnet = NNClassifier( Xtrain.shape[1], [100, 250, 250, 100], len(np.unique(Ttrain)), activation_function='relu', device=device)

nnet.train(Xtrain, Ttrain, 1000, batch_size=200, learning_rate=0.005, method='adam', verbose=True)

adam: Epoch 100 Loss 0.001
adam: Epoch 200 Loss 0.047
adam: Epoch 300 Loss 0.001
adam: Epoch 400 Loss 0.001
adam: Epoch 500 Loss 0.001
adam: Epoch 600 Loss 0.003
adam: Epoch 700 Loss 0.001
adam: Epoch 800 Loss 0.002
adam: Epoch 900 Loss 0.050
adam: Epoch 1000 Loss 0.003


NNClassifier(
  (fc_layers): ModuleList(
    (0): Linear(in_features=57, out_features=100, bias=True)
    (1): Linear(in_features=100, out_features=250, bias=True)
    (2): Linear(in_features=250, out_features=250, bias=True)
    (3): Linear(in_features=250, out_features=100, bias=True)
    (4): Linear(in_features=100, out_features=10, bias=True)
  )
)

In [9]:
confusion_matrix(nnet.use(Xtest)[0], Ttest)

Percent Correct  90.55276381909547


,blues,classical,country,disco,hiphop,jazz,metal,pop,reggae,rock
blues,93.0,0.0,4.0,0.0,1.0,2.0,0.0,0.0,0.0,0.0
classical,0.0,96.0,1.0,0.0,0.0,2.0,0.0,0.0,0.0,1.0
country,3.0,0.0,87.9,1.0,0.0,2.0,0.0,0.0,3.0,3.0
disco,0.0,1.0,2.0,87.9,3.0,0.0,1.0,1.0,1.0,3.0
hiphop,1.0,0.0,0.0,1.0,90.9,0.0,2.0,2.0,3.0,0.0
jazz,0.0,2.0,2.0,2.0,0.0,94.0,0.0,0.0,0.0,0.0
metal,0.0,0.0,0.0,1.0,2.0,0.0,93.0,0.0,0.0,4.0
pop,0.0,0.0,1.0,2.0,1.0,0.0,0.0,90.0,4.0,2.0
reggae,2.0,0.0,1.0,3.0,5.0,0.0,0.0,1.0,88.0,0.0
rock,2.0,0.0,3.0,2.0,2.0,1.0,2.0,1.0,2.0,84.8
